In [1]:
import pandas as pd
import folium
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
from folium.plugins import HeatMap
from global_land_mask import globe
from collections import defaultdict
import geopandas as gpd
import pygeohash as pgh
import seaborn as sns
from shapely.geometry import Polygon
import branca.colormap
from matplotlib import rcParams
from functools import reduce

In [2]:
x = pd.read_csv("2018_Conainer_visData.csv")[["imo","hour","long_x","lat_x","Size_Bin",'EM_CO2',"EM_CO2_LNG"]]

In [3]:
x = x[x["hour"]<745][x["long_x"]!= -99999.0][x["lat_x"]!= -99999.0]
x

<ipython-input-3-ea7ed866ecf4>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x = x[x["hour"]<745][x["long_x"]!= -99999.0][x["lat_x"]!= -99999.0]
<ipython-input-3-ea7ed866ecf4>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x = x[x["hour"]<745][x["long_x"]!= -99999.0][x["lat_x"]!= -99999.0]


,imo,hour,long_x,lat_x,Size_Bin,EM_CO2,EM_CO2_LNG
35040,7617890,0,-139.536276,27.388975,3,1.427435e+07,1.260580e+07
35041,7617890,1,-139.043204,27.545672,3,1.482776e+07,1.309452e+07
35042,7617890,2,-138.731787,27.644878,3,1.517669e+07,1.340266e+07
35043,7617890,3,-138.562793,27.712152,3,1.427435e+07,1.260580e+07
35044,7617890,4,-138.259306,27.914180,3,1.471434e+07,1.299436e+07
...,...,...,...,...,...,...,...
45097355,9843194,740,120.909735,31.852588,1,4.893651e+05,4.321625e+05
45097356,9843194,741,120.909763,31.852387,1,4.893651e+05,4.321625e+05
45097357,9843194,742,120.909762,31.852267,1,4.893651e+05,4.321625e+05
45097358,9843194,743,120.909717,31.852131,1,4.893651e+05,4.321625e+05


In [6]:
#plot markers for the top 10 bunkering ports on the map
def addBunker(vis):
    folium.Marker(location=[1.3521,103.8198],popup='Singapore',tooltip='Singapore',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[51.949,4.153],popup='Rotterdam',tooltip='Rotterdam',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[25.175,56.3586],popup='Fujairah',tooltip='Fujairah',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[22.2932,114.1613],popup='Hong Knog',tooltip='Hong Kong',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[51.2411,4.403],popup='Antwerp',tooltip='Antwerp',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[35.103,129.0424],popup='Busan',tooltip='Busan',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[36.133,-5.35],popup='Gibraltar',tooltip='Gibraltar',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[8.9575,-79.561],popup='Panama',tooltip='Panama',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[36.1252,-5.4395],popup='Algeciras',tooltip='Algeciras',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[33.740,-118.281],popup='Los Angeles',tooltip='Los Angeles',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    
    return vis
#For each lat,long pair in the dataset, calculaate geohash values and return the centroid of the geohash blocks
# More info on geohash: https://www.movable-type.co.uk/scripts/geohash.html
def getCentroid(x):
    l=x.split(',')
    lat=float(l[0])
    lon=float(l[1])
    prec=4
    gh=pgh.encode(lat,lon,precision=prec)
    return pgh.decode(gh)
sbins=[1,2,3,4,5,6,7,8,9]
#Check if a lat, long pair is on land, returns True/False
def isinLand(x):
    #l=x.split(',')
    lat=float(x[0])
    lon=float(x[1])
    return globe.is_land(lat,lon)

In [7]:
x_plot=x
x_plot

,imo,hour,long_x,lat_x,Size_Bin,EM_CO2,EM_CO2_LNG
35040,7617890,0,-139.536276,27.388975,3,1.427435e+07,1.260580e+07
35041,7617890,1,-139.043204,27.545672,3,1.482776e+07,1.309452e+07
35042,7617890,2,-138.731787,27.644878,3,1.517669e+07,1.340266e+07
35043,7617890,3,-138.562793,27.712152,3,1.427435e+07,1.260580e+07
35044,7617890,4,-138.259306,27.914180,3,1.471434e+07,1.299436e+07
...,...,...,...,...,...,...,...
45097355,9843194,740,120.909735,31.852588,1,4.893651e+05,4.321625e+05
45097356,9843194,741,120.909763,31.852387,1,4.893651e+05,4.321625e+05
45097357,9843194,742,120.909762,31.852267,1,4.893651e+05,4.321625e+05
45097358,9843194,743,120.909717,31.852131,1,4.893651e+05,4.321625e+05


In [8]:
x_plot['location']=x_plot.lat_x.astype(str)+','+x_plot.long_x.astype(str)
x_plot.location=x_plot.location.apply(lambda x: getCentroid(x))
x_plot=x_plot.groupby(['location','Size_Bin']).agg({'EM_CO2':'sum','EM_CO2_LNG':'sum'})
x_plot.reset_index(inplace=True)
x_plot['inLand']=x_plot.location
print('isinLand')

x_plot.inLand=x_plot.inLand.apply(lambda x:isinLand(x))
x_plot=x_plot[x_plot.inLand==False]

isinLand


In [9]:
x_plot=x_plot.iloc[:,:-1]

In [10]:
x_plot

,location,Size_Bin,EM_CO2,EM_CO2_LNG
4,"(-71.0, -8.0)",1,3.083635e+07,2.723184e+07
7,"(-70.0, -9.0)",1,8.314751e+05,7.342827e+05
8,"(-70.0, -8.0)",1,3.393004e+07,2.996391e+07
9,"(-70.0, -7.0)",1,8.629327e+06,7.620632e+06
10,"(-70.0, -6.0)",1,3.908668e+06,3.451778e+06
...,...,...,...,...
60871,"(87.0, 17.0)",6,3.852675e+07,3.402330e+07
60872,"(87.0, 33.0)",6,3.852675e+07,3.402330e+07
60873,"(87.0, 47.0)",4,2.576232e+07,2.275092e+07
60874,"(87.0, 47.0)",6,3.852675e+07,3.402330e+07


In [11]:
dfs=[]
for i in sbins:
    dfs.append(x_plot[x_plot.Size_Bin==i])
    dfs[i-1].columns=['location', 'Size_Bin', 'EM_CO2_'+str(i), 'EM_CO2_LNG_'+str(i)]
    #dfs[i-1]['plot_'+str(i)]=0
    del dfs[i-1]['Size_Bin']
    dfs[i-1]['EM_CO2_Electricity_Hydrogen_'+str(i)]=0
    #print(dfs[i-1].columns)

<ipython-input-11-bfa2d44d621c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs[i-1]['EM_CO2_Electricity_Hydrogen_'+str(i)]=0
<ipython-input-11-bfa2d44d621c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs[i-1]['EM_CO2_Electricity_Hydrogen_'+str(i)]=0
<ipython-input-11-bfa2d44d621c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [12]:
#Join all the dataframe in dfs 
df=reduce(lambda l,r:pd.merge(l,r,on=['location'],how='outer'),dfs).fillna(0)

In [13]:
df

,location,EM_CO2_1,EM_CO2_LNG_1,EM_CO2_Electricity_Hydrogen_1,EM_CO2_2,EM_CO2_LNG_2,EM_CO2_Electricity_Hydrogen_2,EM_CO2_3,EM_CO2_LNG_3,EM_CO2_Electricity_Hydrogen_3,...,EM_CO2_Electricity_Hydrogen_6,EM_CO2_7,EM_CO2_LNG_7,EM_CO2_Electricity_Hydrogen_7,EM_CO2_8,EM_CO2_LNG_8,EM_CO2_Electricity_Hydrogen_8,EM_CO2_9,EM_CO2_LNG_9,EM_CO2_Electricity_Hydrogen_9
0,"(-71.0, -8.0)",3.083635e+07,2.723184e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"(-70.0, -9.0)",8.314751e+05,7.342827e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"(-70.0, -8.0)",3.393004e+07,2.996391e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"(-70.0, -7.0)",8.629327e+06,7.620632e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"(-70.0, -6.0)",3.908668e+06,3.451778e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18340,"(11.0, -47.0)",0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.009552e+07,5.307087e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18341,"(11.0, -46.0)",0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.568151e+07,1.384847e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18342,"(12.0, -48.0)",0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8.687912e+07,7.672369e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18343,"(12.0, -47.0)",0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.409279e+07,1.244546e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
df['EM_plot']=0

In [15]:
fname='Jan 2018 Emissions.html'

In [16]:
plot=folium.Map(location=[1.3521,103.8198])
plot.fit_bounds([(10.66017,85.865),(-4.07,85.865),(-4.07,119.21),(10.6607,119.21)])
#plot
min_lat=-4.07
min_lon=85.865
max_lat=10.6607
max_lon=119.21


In [17]:
#Format for args in web app
args={2: 'Assigned Fuel', 3: 'LNG', 6: 'Hydrogen'}

In [18]:
for i in args:
    if args[i]=='Assigned Fuel':
        df['EM_plot']+=df['EM_CO2_'+str(i)]
    elif args[i]=='LNG':
        df['EM_plot']+=df['EM_CO2_LNG_'+str(i)]

In [19]:
plot_data=[]
locs=df.location.to_list()
co2=df.EM_plot.to_list()
print('Making Map')
plot_data=[]
all_co2_max=max(co2)/(10**6)
all_co2_min=min(co2)/(10**6)
for i in range(len(locs)):
    plot_data.append([locs[i][0],locs[i][1],co2[i]/(10**6)])
gradient_map={0.1:'blue',0.3:'cyan', 0.5:'lime', 0.75:'yellow',0.9:'red'}
lst=[0.1,0.3,0.5,0.75,0.9]
colormap = branca.colormap.LinearColormap(colors=['blue','cyan', 'lime', 'yellow','red']).scale(all_co2_min,all_co2_max).to_step(5)
colormap.caption = 'Jan 2018 Co2 Emissions (Tonnes)'
Intensity = HeatMap(plot_data,radius = 15,blur=20,min_opacity=0.3,gradient=gradient_map)
Intensity.add_to(plot)
plot=addBunker(plot)
plot.add_child(colormap)
plot.save(fname)
fname

Making Map


'Jan 2018 Emissions.html'